In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 30.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=8b445d832afa31f07298ba43a3335a6553e5ca075d1cb1a9f00ce9c6599f3eb7
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
session = SparkSession.builder.appName("facebook").getOrCreate()
data = session.read.csv("Facebook_data.csv", header = True, inferSchema = True)

In [ ]:
data.show(4)

+----+----------+--------+------------+---------+------+---+-----------------+---------------------+--------+-------------+----------------+-----------------+------------+
|S.No|FacebookId| Surname|FBValueScore|Geography|Gender|Age|On FB Since Years|Spend on FB Till Date|NumOfAds|HasCreditCard|IsActiveBusiness|EstimatedTurnover|ExitedFromFB|
+----+----------+--------+------------+---------+------+---+-----------------+---------------------+--------+-------------+----------------+-----------------+------------+
|   1|  15565701|   Ferri|         698|    Spain|Female| 39|                9|            161993.89|       1|            0|               0|         90212.38|           0|
|   2|  15565706|Akobundu|         612|    Spain|  Male| 35|                1|                  0.0|       1|            1|               1|         83256.26|           1|
|   3|  15565714|Cattaneo|         601|   France|  Male| 47|                1|             64430.06|       2|            0|               1|

In [ ]:
from pyspark.ml.feature import StringIndexer
str_indx = StringIndexer(inputCols=["Geography", "Gender"], outputCols = ["newGeography", "newGender"])

In [ ]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols = ["newGeography", "newGender", 'FacebookId','FBValueScore','Age','On FB Since Years','Spend on FB Till Date','NumOfAds','HasCreditCard','IsActiveBusiness','EstimatedTurnover','ExitedFromFB'], outputCol="allfeatures")


In [ ]:
from pyspark.ml.classification import LogisticRegression
model = LogisticRegression(featuresCol = 'allfeatures', labelCol = 'ExitedFromFB')

In [ ]:
from pyspark.ml import Pipeline
mypipline = Pipeline(stages = [str_indx, assembler, model])

In [ ]:
training, test = data.randomSplit([0.8, 0.2])

In [ ]:
fitmodel = mypipline.fit(training)

In [ ]:
result = fitmodel.transform(test)

In [ ]:
result.show()

+----+----------+---------------+------------+---------+------+---+-----------------+---------------------+--------+-------------+----------------+-----------------+------------+------------+---------+--------------------+--------------------+--------------------+----------+
|S.No|FacebookId|        Surname|FBValueScore|Geography|Gender|Age|On FB Since Years|Spend on FB Till Date|NumOfAds|HasCreditCard|IsActiveBusiness|EstimatedTurnover|ExitedFromFB|newGeography|newGender|         allfeatures|       rawPrediction|         probability|prediction|
+----+----------+---------------+------------+---------+------+---+-----------------+---------------------+--------+-------------+----------------+-----------------+------------+------------+---------+--------------------+--------------------+--------------------+----------+
|  17|  15566251|        Ferrari|         618|   France|Female| 37|                5|             96652.86|       1|            1|               0|          98686.4|       

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
eval_obj = BinaryClassificationEvaluator(rawPredictionCol = 'rawPrediction', labelCol = "ExitedFromFB")
accuracy = eval_obj.evaluate(result)
print(accuracy)

0.9999992798087172
